### Introduction to Machine Learning: Clustering (Zillow)
#### Corey Solitaire
#### 10/09/2020

In [1]:
import acquire
import prepare
import wrangle_zillow
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from scipy import stats
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, SelectKBest, RFE 
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

## Acquire & Summarize

### 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

    - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
    - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
    - Only include properties that include a latitude and longitude value.

In [2]:
# Acquire df
df = acquire.get_zillow_data(cached=False)
#pd.set_option('display.max_rows', df.shape[0]+1) # Shows all the rows

In [3]:
df.shape

(77381, 70)

In [4]:
# Rename duplicate column id/id with id_delete/id
def rename_columns(df):
    df.columns = ['parcelid', 'typeconstructiontypeid', 'storytypeid',
           'propertylandusetypeid', 'heatingorsystemtypeid', 'buildingclasstypeid',
           'architecturalstyletypeid', 'airconditioningtypeid', 'id_delete',
           'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
           'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
           'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
           'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
           'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
           'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
           'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
           'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
           'propertycountylandusecode', 'propertyzoningdesc',
           'rawcensustractandblock', 'regionidcity', 'regionidcounty',
           'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
           'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
           'numberofstories', 'fireplaceflag', 'structuretaxvaluedollarcnt',
           'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
           'taxamount', 'taxdelinquencyflag', 'taxdelinquencyyear',
           'censustractandblock', 'id', 'logerror', 'pid', 'tdate',
           'airconditioningdesc', 'architecturalstyledesc', 'buildingclassdesc',
           'heatingorsystemdesc', 'propertylandusedesc', 'storydesc',
           'typeconstructiondesc']
    df.drop(columns = ['id_delete','pid'], inplace = True)
    df = df.drop(df.index[[77380], inplace = True]
    return df


SyntaxError: invalid syntax (<ipython-input-4-8ff76cbeae0b>, line 26)

In [ ]:
rename_columns(df)

In [ ]:
df.shape

In [ ]:
# Delete duplicate columns id_delete and pid (dupilcate of parcelid)
#df.drop(columns = ['id_delete','pid'], inplace = True)
#df.shape

***

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

***

In [ ]:
def df_summary(df):
    print('The shape of the df:') 
    print(df.shape)  # df shape (row/column)
    print('\n')
    print('Columns, Non-Null Count, Data Type:')
    print(df.info())      # Column, Non Null Count, Data Type
    print('\n')
    print('Summary statistics for the df:') 
    print(df.describe())             # Summary Statistics on Numeric Data Types
    print('\n')
    print('Number of NaN values per column:') 
    print(df.isna().sum())           # NaN by column
    print('\n')
    print('Number of NaN values per row:')  
    print(df.isnull().sum(axis=1))   # NaN by row
    print('\n')
    print('Value Counts per Column:')
    for col in df.columns:
        print('-' * 40 + col + '-' * 40 , end=' - ')
        display(df[col].value_counts(dropna=False).head(10))
        #display(df_resp[col].value_counts())  # Displays all Values, not just First 10

In [ ]:
df_summary(df)

***

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.*

***

In [ ]:
def count_and_percent_missing_row(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    total_missing = df.isnull().sum()
    missing_value_df = pd.DataFrame({'num_rows_missing': total_missing,
                                     'pct_rows_missing': percent_missing})
    return missing_value_df

In [ ]:
count_and_percent_missing_row(df)

***

#### Based on this I would want to delete attributes that have less than 50% of their data missing.

***

***

### 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

***

In [ ]:
def count_and_percent_missing_column(df):
    num_rows = df.loc[:].isnull().sum()
    num_cols_missing = df.loc[:, df.isna().any()].count()
    pct_cols_missing = round(df.loc[:, df.isna().any()].count() / len(df.index) * 100, 3)
    missing_cols_and_rows_df = pd.DataFrame({'num_cols_missing': num_cols_missing,
                                             'pct_cols_missing': pct_cols_missing,
                                             'num_rows': num_rows})
    missing_cols_and_rows_df = missing_cols_and_rows_df.fillna(0)
    missing_cols_and_rows_df['num_cols_missing'] = missing_cols_and_rows_df['num_cols_missing'].astype(int)
    return missing_cols_and_rows_df

In [ ]:
count_and_percent_missing_column(df)



#### Based on this I would want to delete columns that have less than 75% of their data missing.

***

## Prepare

### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.


Property Codes that Fit the Breif of (Single Unit Properties)

propertylandusetypeid
-        261          Single Family Residential
-        263          Mobile Home
-        264          Townhouse
-        266          Condominium
-        273          Bungalow
-        276          Patio Home
-        275          Manufactured, Modular, Prefabricated Homes
-        279          Inferred Single Family Residential

Alternate Methods to determine single Unit Properties

df.unitcnt == 1   
df.roomcnt > 0   
df.garagecarcnt > 0   
df.bedroomcnt > 0   
df.bathroomcnt > 0   

In [ ]:
df.shape

In [ ]:
df = acquire.get_zillow_data(cached=True)
unit_df = df[df['unitcnt']==1]
unit_df.shape

In [ ]:
df = acquire.get_zillow_data(cached=True)
room_df = df[df['roomcnt']>0]
room_df.shape

In [ ]:
df = acquire.get_zillow_data(cached=True)
garage_df = df[df['garagecarcnt']>0]
garage_df.shape

In [ ]:
df = acquire.get_zillow_data(cached=True)
bed_df = df[df['bedroomcnt']>0]
bed_df.shape

In [ ]:
df = acquire.get_zillow_data(cached=True)
bath_df = df[df['bathroomcnt']>0]
bath_df.shape

In [ ]:
# Unique dataframe that does not use proptertylandusetypeid to identify single unit properties
seconday_featutes_df = pd.concat([bed_df, bath_df, garage_df, room_df, unit_df]).drop_duplicates('id').reset_index(drop=True)
seconday_featutes_df.shape

In [ ]:
#pd.set_option('display.max_rows', df.shape[0]+1) # Shows all the rows
#new_df.parcelid.value_counts(ascending=False)

In [ ]:
# Unique dataframe tha uses propterylandusetypeid to identify single unit properties
df = acquire.get_zillow_data(cached=True)
p261_df = df[df['propertylandusetypeid'] == 261]
p263_df = df[df['propertylandusetypeid'] == 263]
p264_df = df[df['propertylandusetypeid'] == 264]    
p266_df = df[df['propertylandusetypeid'] == 266] 
p273_df = df[df['propertylandusetypeid'] == 273]
p275_df = df[df['propertylandusetypeid'] == 275]
p276_df = df[df['propertylandusetypeid'] == 276]    
p279_df = df[df['propertylandusetypeid'] == 279]
property_df = pd.concat([p261_df, p263_df, p264_df, p266_df, p273_df, p275_df, p276_df, p279_df]).drop_duplicates('id').reset_index(drop=True)
property_df.shape

In [ ]:
# Combined Dataframe, seconday_features_df and verified property_df
all_single_unit_properties_df = pd.concat([seconday_featutes_df, property_df]).drop_duplicates('id').reset_index(drop=True)
all_single_unit_properties_df.shape

In [ ]:
# Check for duplicated rows
duplicate = all_single_unit_properties_df[all_single_unit_properties_df.duplicated(keep = 'last')]   
print("Duplicate Rows :") 
duplicate

In [ ]:
df = df = pd.read_csv('zillow_df.csv')
unit_df = df[df['unitcnt']==1]
df = df = pd.read_csv('zillow_df.csv')
room_df = df[df['roomcnt']>0]
df = df = pd.read_csv('zillow_df.csv')
garage_df = df[df['garagecarcnt']>0]
df = df = pd.read_csv('zillow_df.csv')
bed_df = df[df['bedroomcnt']>0]
df = df = pd.read_csv('zillow_df.csv')
bath_df = df[df['bathroomcnt']>0]
df = df = pd.read_csv('zillow_df.csv')
#seconday_featutes_df = pd.concat([bed_df, bath_df, garage_df, room_df, unit_df]).drop_duplicates('id').reset_index(drop=True)
df = df = pd.read_csv('zillow_df.csv')
p261_df = df[df['propertylandusetypeid'] == 261]
df = df = pd.read_csv('zillow_df.csv')
p263_df = df[df['propertylandusetypeid'] == 263]
df = df = pd.read_csv('zillow_df.csv')
p264_df = df[df['propertylandusetypeid'] == 264]    
df = df = pd.read_csv('zillow_df.csv')
p266_df = df[df['propertylandusetypeid'] == 266] 
df = df = pd.read_csv('zillow_df.csv')
p273_df = df[df['propertylandusetypeid'] == 273]
df = df = pd.read_csv('zillow_df.csv')
p275_df = df[df['propertylandusetypeid'] == 275]
df = df = pd.read_csv('zillow_df.csv')
p276_df = df[df['propertylandusetypeid'] == 276]    
df = df = pd.read_csv('zillow_df.csv')
p279_df = df[df['propertylandusetypeid'] == 279]
complete_df = pd.concat([p261_df, p263_df, p264_df, p266_df, p273_df, p275_df, p276_df, p279_df,bed_df, bath_df, garage_df, room_df, unit_df]).drop_duplicates('id').reset_index(drop=True)
df = pd.concat([complete_df]).drop_duplicates('id').reset_index(drop=True)
   

In [ ]:
print(df.shape)
duplicate = df[df.duplicated(keep = 'last')]   
print("Duplicate Rows :") 
duplicate


#### This suggests that filtering by propertylandusetypeid over filtered the dataframe.  By combining the two filters wer were able to add 6,000 more records to our study

***

### 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

***

**The input:**   

 - A dataframe
 - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
 - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
        
        
**The output:**   

- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
       
- hint:
  - Look up the dropna documentation.
  - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
  - Make use of inplace, i.e. inplace=True/False.



### Prep Data Function:

In [ ]:
# Function to Prep Data
def data_prep(df, cols_to_remove=[], prop_required_column=.5, prop_required_row=.75):
    
    def remove_columns(df, cols_to_remove):  
        df = df.drop(columns=cols_to_remove)
        return df

    def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
        threshold = int(round(prop_required_column*len(df.index),0))
        df.dropna(axis=1, thresh=threshold, inplace=True)
        threshold = int(round(prop_required_row*len(df.columns),0))
        df.dropna(axis=0, thresh=threshold, inplace=True)
        return df
    
    df = remove_columns(df, cols_to_remove)  # Removes Specified Columns
    df = handle_missing_values(df, prop_required_column, prop_required_row) # Removes Specified Rows
    #df.dropna(inplace=True) # Drops all Null Values From Dataframe
    return df


# How to Call the Function
# df = prepare.data_prep(
#     df,
#     cols_to_remove=[],
#     prop_required_column=.6,
#     prop_required_row=.75
# )

In [ ]:
df = prepare.data_prep(
    all_single_unit_properties_df,
    cols_to_remove=[],
    prop_required_column=.6,
    prop_required_row=.75
)
df.head()

In [ ]:
df.info()

### 3. Decide how to handle the remaining missing values:

- Fill with constant value.
- Impute with mean, median, mode.
- Drop row/column

*** For simplicitys sake I will choose to drop the remaning NaN values, and will do that in the function.***

### 4. Add Above Functions to File Called wrangle_zillow.py

In [ ]:
# df_summary(provides df info before wrangle function)
df = df = pd.read_csv('zillow_df.csv')
wrangle_zillow.df_summary(df)

In [ ]:
# Testing wrangle_zillow Function (Splits/Scales)

df, X_train_explore, X_train_scaled, X_validate_scaled, X_test_scaled = wrangle_zillow.wrangle_zillow(acquire.get_zillow_data(cached=False))

In [ ]:
df.shape, X_train_explore.shape, X_train_scaled.shape, X_validate_scaled.shape, X_test_scaled.shape 

In [ ]:
# Testing IQR Function
wrangle_zillow.quartiles_and_outliers(X_train_explore)

### Visualizing and Determining IQR

In [ ]:
# Visualize Data
df.hist(figsize=(24, 10), bins=20)
plt.show()

In [ ]:
def quartiles_and_outliers(df):
    def get_upper_outliers(s, k):
        '''
        Given a series and a cutoff value, k, returns the upper outliers for the
        series.

        The values returned will be either 0 (if the point is not an outlier), or a
        number that indicates how far away from the upper bound the observation is.
        '''
        q1, q3 = s.quantile([.25, .75])
        iqr = q3 - q1
        upper_bound = q3 + k * iqr
        return s.apply(lambda x: max([x - upper_bound, 0]))

    def add_upper_outlier_columns(df, k):
        '''
        Add a column with the suffix _outliers for all the numeric columns
        in the given dataframe.
        '''
        # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
        #                 for col in df.select_dtypes('number')}
        # return df.assign(**outlier_cols)
        for col in df.select_dtypes('number'):
            df[col + '_outliers'] = get_upper_outliers(df[col], k)
        return df

    add_upper_outlier_columns(df, k=1.5)    
    outlier_cols = [col for col in df if col.endswith('_outliers')]
    for col in outlier_cols:
        print('~~~\n' + col)
        data = df[col][df[col] > 0]
        print(data.describe())

In [ ]:
quartiles_and_outliers(df)

In [ ]:
df.info()